In [5]:
%pip install -r requirements.txt --user

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: C:\Users\mosas\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from code_completion_lib.imports.imports import Imports
from code_completion_lib.parse_notebooks import Parser
from code_completion_lib.methods.find_methods_in_code import Methods
from code_completion_lib.logger.logger import Logger
from code_completion_lib.code_completion import CodeCompletion
import os
from math import log2

In [7]:
logger = Logger(__name__, mode="a")
logger.info("-----------------------------------------------------")

In [8]:
size = ["small","medium","big"]
path = "C:\data\data_parsed"

In [ ]:
try:
    parcer = Parser(r'C:\data\notebooks', r'C:\data\data_parsed', logger=logger)
    parcer.parse()
    parcer.check_language()
except Exception:
    logger.error("Exception")


In [3]:
for name in size:
    try:
        imports = Imports(os.path.join(path, name),size=name,logger=logger)
        imports.process()
    except Exception:
        logger.error("Exception")

In [ ]:
for name in size:
    try:
        completion = CodeCompletion(size=name, logger=logger)
        completion.import_clusterization()
    except Exception:
        logger.error("Exception")

In [3]:
try:
    completion = CodeCompletion(size="small", logger=logger)
    completion.import_clusterization()
except Exception:
    logger.error("Exception")

In [ ]:
for name in size:
    model_path = rf'code_completion_lib\models\{name}'
    try:
        methods = Methods(os.path.join(path, name),size=name, logger=logger)
        methods.find_methods(model_path)
    except Exception:
            logger.error("Exception")
    break

In [7]:
def top_3(y_true, y_pred):
    true = 0
    length = len(y_pred)
    if length == 0:
        return 0
    for i in range(len(y_pred)):
        for j in range(len(y_pred[i])):
            if j==3:
                continue
            if y_pred[i][j] == y_true[i]:
                true+=1
    return true/length

In [8]:
def nDCG(y_true, y_pred):
    relevance = []
    length = len(y_pred)
    if length == 0:
        return 0
    for i in range(len(y_pred)):
        relevance.append([])
        for j in range(3):
            if j >= len(y_pred[i]):
                relevance[i].append(0)
            elif y_pred[i][j] == y_true[i]:
                relevance[i].append(1)
            else:
                relevance[i].append(0)
    ndcg = 0
    for element in relevance:
        ideal = element.copy()
        ideal.sort(reverse=True)
        dcg = 0
        idcg = 0
        for i in range(len(element)):
            dcg += element[i]/log2(i+2)
            idcg += ideal[i]/log2(i+2)
        if idcg != 0:
            ndcg += dcg/idcg
    ndcg /= len(relevance)
    return ndcg

In [9]:
import time
for name in size:
    print(f"size: {name}")
    dataset = pd.read_csv(rf'code_completion_lib\methods\models\{name}\data.csv')
    models = dataset.keys()[2:]
    for model in models:
        print(model)
        try:
            X = dataset[["varible_name", model]]
            y = dataset[["method"]]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
            completion = CodeCompletion(size=name, logger=logger)
            completion.relations_variable_with_method(X_train, y_train, model=model)
            completion.relations_cluster_with_variable(X_train, y_train, model=model)


            y_pred_top1 = []
            y_pred_top3 = []
            y_true = y_test['method'].values.tolist()
            for index in range(X_test.shape[0]):
                f_completion = completion.get_function_completion(model=model, variable_name=X_test.values[index][0],cluster=X_test.values[index][1],number=3)
                y_pred_top1.append(f_completion[0])
                y_pred_top3.append(f_completion)


            acc_top1 = accuracy_score(y_true, y_pred_top1)
            acc_top3 = top_3(y_true, y_pred_top3)
            ndcg = nDCG(y_true, y_pred_top3)

            print(f"top 1 accuracy: {acc_top1}")
            print(f"top 3 accuracy: {acc_top3}")
            print(f"ndcg: {ndcg}\n")

        except Exception:
            logger.error("Exception")
    break


size: small
AffinityPropagation(damping=0.999, random_state=0)
top 1 accuracy: 0.5088967971530249
top 3 accuracy: 0.5444839857651246
ndcg: 0.5304179289273012

AgglomerativeClustering(metric='euclidean', n_clusters=13)
top 1 accuracy: 0.505338078291815
top 3 accuracy: 0.5444839857651246
ndcg: 0.5291045116802244

KMeans(n_clusters=13, n_init='auto', random_state=0)
top 1 accuracy: 0.5088967971530249
top 3 accuracy: 0.5409252669039146
ndcg: 0.5281726273131678

MeanShift(bandwidth=2.3)
top 1 accuracy: 0.4804270462633452
top 3 accuracy: 0.5729537366548043
ndcg: 0.5341454663955276

SpectralClustering(assign_labels='discretize', n_clusters=13, random_state=0)
top 1 accuracy: 0.49466192170818507
top 3 accuracy: 0.5480427046263345
ndcg: 0.5274095615531277

